In [0]:
#Importing all the required libraries
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.metrics import precision_recall_fscore_support

Using TensorFlow backend.


In [0]:
#Importing the given data, Seperated the features and target from train and test data.
train_features = np.loadtxt('train_data.txt', delimiter = ',', usecols = (0,1))
train_target = np.loadtxt('train_data.txt', delimiter = ',', usecols = 2)
test_features = np.loadtxt('test_data.txt', delimiter = ',', usecols = (0,1))
test_target = np.loadtxt('test_data.txt', delimiter = ',', usecols = 2)

In [0]:
#sigmoid function
def sigmoid(x):
 
    m = np.exp(-x)
    return(1.0/(1.0 + m))



#differentaited sigmoid function in matrix form
def der_sigmoid(x):
    k = np.zeros((x.shape[0], x.shape[0]))
    for i in range(len(k)):
        for j in range(len(k)):
            if i == j:
                k[i][j] = sigmoid(x[i]) *(1-sigmoid(x[i]))
            else: 
                k[i][j] = 0
    return(k)

#Softmax function
def softmax(x):
    m = 0
    for i in range(len(x)):
        m = m + np.exp(x[i])
    for i in range(len(x)):
        x[i] = np.exp(x[i])/m
    return(x)

#differentiated softmax function, in jacobian format.
def der_soft(x):
    k = np.zeros((x.shape[0], x.shape[0]))
    for i in range(x.shape[0]):
        for j in range(x.shape[0]):
            if i == j:
                k[i][j] = x[i] *(1-x[i])
            else: 
                k[i][j] = -x[i]*x[j]
    return(k)


#Initalising the weights
def init_weights():
    w1 = np.random.rand(3,2)
    w2 = np.random.rand(3,3)
    w3 = np.random.rand(2,3)
    return([w1,w2,w3]) 
    
#forward calculation
def feedforward(x, w1, w2, w3):
    a1 = sigmoid(np.dot(w1,x.T))
    a2 = sigmoid(np.dot(w2,a1))
    y = softmax(np.dot(w3,a2))
    
    return(y)

#function to set right the given target values form the text data.
# took '0' and '1' as two classes. if the value is '1' in text data, then the groundtruth will be [0, 1]
#and for '0', it's reverse
def y_out(x):
    if x == 1:
        return(np.matrix([0, 1]))
    else:
        return(np.matrix([1, 0]))
    
#function to train the network
def train(x, y):
    [w1,w2,w3] = init_weights()
    n = 0.01 #learning rate
    LOSS = []
    for i in range(2): #number of epochs
        
        for j in range(len(x)): #iteration through number of samples 
            
          
            #forward calculation in the network
            z1 = np.dot(w1,np.matrix(x[j]).T)
            
            a1 = sigmoid(z1)
            z2 = np.dot(w2,a1)
            a2 = sigmoid(z2)
            z3 = np.dot(w3,a2)
         
            y_pr = np.matrix(softmax(z3))
              
            
            m = y[j]
            k = y_out(m).T
            
            
            #calculating the loss of the network
            loss = np.sum(np.square((k - y_pr)))
            
            #change in loss
            del_loss = -2*((k - y_pr))
           
            
            
            
            
            del3 = np.dot(der_soft(y_pr), del_loss)
            del2 = np.dot(np.dot(der_sigmoid(z2), w3.T), del3)
            del1 = np.dot(np.dot(der_sigmoid(z1), w2.T), del2)
            
            #change in weights is given by below
            del3_w = np.dot(np.matrix(a2),np.matrix(del3).T)
            del2_w = np.dot(np.matrix(a1), np.matrix(del2).T)
            del1_w = np.dot(np.matrix(x[j]).T, np.matrix(del1).T)
            
            #updation of weights
            w1 = w1 - n*del1_w.T
            w2 = w2 - n*del2_w.T
            w3 = w3 - n*del3_w.T
            
          
        LOSS.append(loss)
        print('The loss after ' + str(i+1) + ' epochs : ', loss)
    
    print('----------------------------------------------------------------')
    print('The weight w1 after ' + str(i+1) + ' epochs:', w1)
    print('The shape of w1 :', w1.shape)
    print('The weight w2 after ' + str(i+1) + ' epochs:', w2)
    print('The shape of w2 : ', w2.shape)
    print('The weight w3 after ' + str(i+1) + ' epochs:', w3)
    print('The shape of w3 :', w3.shape)
    return([w1, w2, w3, LOSS])
  

In [0]:
y = train_target
x = train_features

In [0]:
[w1, w2, w3, LOSS] = train(x,y)

The loss after 1 epochs :  0.4922746936378984
The loss after 2 epochs :  0.4487008217852561
----------------------------------------------------------------
The weight w1 after 2 epochs: [[ 0.42386352  1.03264539]
 [ 0.80218312 -0.13311215]
 [ 0.10019367  0.80649237]]
The shape of w1 : (3, 2)
The weight w2 after 2 epochs: [[ 0.03525057  1.26829459 -0.08213004]
 [ 0.25954633  0.42943571  0.66759723]
 [ 0.57562927  0.14066945  0.76580635]]
The shape of w2 :  (3, 3)
The weight w3 after 2 epochs: [[ 1.24197084  0.69290014 -0.00455182]
 [ 0.09416372  0.60998121  1.14513401]]
The shape of w3 : (2, 3)


In [0]:
def y_mod(x):
    if x == 1:
        return([0, 1])
    else:
        return([1, 0])
k = []
for i in range(len(train_target)):
  k.append(y_mod(train_target[i]))

train_target1 = np.array(k)


In [0]:
x = test_features
NN_result = np.array(feedforward(x, w1, w2, w3))

#function to modify the target values of test data set
def y_mod(x):
    if x == 1:
        return([0, 1])
    else:
        return([1, 0])
j = []
for i in range(len(test_target)):
  j.append(y_mod(test_target[i]))

test_target1 = np.array(j)

In [0]:
#Score test across in-built model without RELU
for i in range(len(NN_result[0])):
  if NN_result[1][i] > NN_result[0][i]:
    NN_result[1][i] = 1
    NN_result[0][i] = 0
  else:
    NN_result[1][i] = 0
    NN_result[0][i] = 1

  k = []
for i in range(len(NN_result[0])):
  k.append([NN_result[0][i], NN_result[1][i]])
test_target_pre = np.array(k)

z = precision_recall_fscore_support(test_target1, test_target_pre)
print('The Precision from in-built model: ', z[0])
print('The Recall from in-built model:', z[1])
print('The Fscore from in-built model: ', z[2])

The Precision from in-built model:  [0.99403579 1.        ]
The Recall from in-built model: [1.    0.994]
The Fscore from in-built model:  [0.99700897 0.99699097]


In [0]:
def relu(x):
  return( x* (x > 0) )

def der_relu(x):
  k = np.zeros((x.shape[0], x.shape[0]))
  for i in range(len(k)):
    for j in range(len(k)):
      if i==j and k[i][0]:
        k[i][j]  = 1
      else: 
        k[i][j] = 0
    return(k)

def train(x, y):
    [w1,w2,w3] = init_weights()
    n = 0.01 #learning rate
    LOSS = []
    l = 1
    #i = i
    while l > 10**-3: 
      #number of epochs
        
        for j in range(len(x)): #iteration through number of samples 
            
          
            #forward calculation in the network
            z1 = np.dot(w1,np.matrix(x[j]).T)
            
            a1 = np.matrix(relu(np.array(z1)))
           
            z2 = np.dot(w2,a1)
            a2 = np.matrix(relu(np.array(z2)))
            z3 = np.dot(w3,a2)
         
            y_pr = np.matrix(softmax(z3))
              
            
            m = y[j]
            k = y_out(m).T
            
            
            #calculating the loss of the network
            loss = np.sum(np.square((k - y_pr)))
            
            #change in loss
            del_loss = -2*((k - y_pr))
           
            
            
            
            
            del3 = np.dot(der_soft(y_pr), del_loss)
            del2 = np.dot(np.dot(der_relu(z2), w3.T), del3)
            del1 = np.dot(np.dot(der_relu(z1), w2.T), del2)
            
            #change in weights is given by below
            del3_w = np.dot(np.matrix(a2),np.matrix(del3).T)
            del2_w = np.dot(np.matrix(a1), np.matrix(del2).T)
            del1_w = np.dot(np.matrix(x[j]).T, np.matrix(del1).T)
            
            #updation of weights
            w1 = w1 - n*del1_w.T
            w2 = w2 - n*del2_w.T
            w3 = w3 - n*del3_w.T
            
          
        LOSS.append(loss)
        print('The loss after  epochs : ', loss)
        l = loss
    #i = i + 1
    
    print('----------------------------------------------------------------')
    print('The weight w1 after  epochs:', w1)
    print('The shape of w1 :', w1.shape)
    print('The weight w2 after  epochs:', w2)
    print('The shape of w2 : ', w2.shape)
    print('The weight w3 after  epochs:', w3)
    print('The shape of w3 :', w3.shape)
    return([w1, w2, w3, LOSS])
  

In [0]:
[w1, w2, w3, LOSS] = train(x,y)

The loss after  epochs :  0.1844807806763029
The loss after  epochs :  0.062198388384854864
The loss after  epochs :  0.028476850645214064
The loss after  epochs :  0.015191899585507
The loss after  epochs :  0.00891325026367388
The loss after  epochs :  0.0055876016016579814
The loss after  epochs :  0.003679802622084254
The loss after  epochs :  0.0025181643750480636
The loss after  epochs :  0.001777192228756365
The loss after  epochs :  0.0012865223562837907
The loss after  epochs :  0.0009514161100198041
----------------------------------------------------------------
The weight w1 after  epochs: [[0.62709354 0.44402638]
 [0.06837613 0.82376136]
 [0.43568491 0.61240641]]
The shape of w1 : (3, 2)
The weight w2 after  epochs: [[0.22484086 0.95403082 0.02536913]
 [0.8042365  0.7012352  0.52336053]
 [0.77122377 0.61202373 0.71876342]]
The shape of w2 :  (3, 3)
The weight w3 after  epochs: [[-1.95171851  1.44332799  1.51791123]
 [ 3.33687792  0.24616508 -0.54788934]]
The shape of w3 : 

In [0]:
x = test_features

def feedforward(x, w1, w2, w3):
    a1 = relu(np.array(np.dot(w1,np.matrix(x).T)))
    a2 = relu(np.array(np.dot(w2,a1)))
    y = softmax(np.dot(w3,a2))

    return(y)

NN_result = np.array(feedforward(x, w1, w2, w3))

#function to modify the target values of test data set
def y_mod(x):
    if x == 1:
        return([0, 1])
    else:
        return([1, 0])
j = []
for i in range(len(test_target)):
  j.append(y_mod(test_target[i]))

test_target1 = np.array(j)


In [0]:
#Score test across in-built model using Relu
for i in range(len(NN_result[0])):
  if NN_result[1][i] > NN_result[0][i]:
    NN_result[1][i] = 1
    NN_result[0][i] = 0
  else:
    NN_result[1][i] = 0
    NN_result[0][i] = 1

  k = []
for i in range(len(NN_result[0])):
  k.append([NN_result[0][i], NN_result[1][i]])
test_target_pre = np.array(k)

z = precision_recall_fscore_support(test_target1, test_target_pre)
print('The Precision from in-built model: ', z[0])
print('The Recall from in-built model:', z[1])
print('The Fscore from in-built model: ', z[2])

The Precision from in-built model:  [0.96899225 1.        ]
The Recall from in-built model: [1.    0.968]
The Fscore from in-built model:  [0.98425197 0.98373984]
